https://adventofcode.com/2023/day/5

In [1]:
input_test = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""

with open('data/day_5.txt') as txtfile:
    input_day = txtfile.read()

In [75]:
import re
import pandas as pd
PROD = 0
input_data = input_day if PROD else input_test
list_maps = ["seed-to-soil map", "soil-to-fertilizer map", "fertilizer-to-water map", "water-to-light map",
             "light-to-temperature map", "temperature-to-humidity map", "humidity-to-location map", ""]
dicc_maps = {"seeds": [int(i) for i in re.findall(r'(?<=seeds:\s)(?=(.*\d+))', input_data)[0].split()]}
for pos, l in enumerate(list_maps[:-1]):
    dicc_maps[l] = [[int(j) for j in i.split()] for i in re.findall(f'(?<={l}:\s)([\d\s]+)(?={list_maps[pos + 1]})', input_data)[0].splitlines() if i]

dicc_maps

{'seeds': [79, 14, 55, 13],
 'seed-to-soil map': [[50, 98, 2], [52, 50, 48]],
 'soil-to-fertilizer map': [[0, 15, 37], [37, 52, 2], [39, 0, 15]],
 'fertilizer-to-water map': [[49, 53, 8], [0, 11, 42], [42, 0, 7], [57, 7, 4]],
 'water-to-light map': [[88, 18, 7], [18, 25, 70]],
 'light-to-temperature map': [[45, 77, 23], [81, 45, 19], [68, 64, 13]],
 'temperature-to-humidity map': [[0, 69, 1], [1, 0, 69]],
 'humidity-to-location map': [[60, 56, 37], [56, 93, 4]]}

# Part 1

In [73]:
def get_destination(value, origins):
    for k, v in origins.items():
        if value in k:
            return value + v
    return value

dicc_origin_ranges = {
    m: {
            pd.Interval(r[1], r[1] + r[2] -1, closed="both"): r[0] - r[1]
            for r in sorted(dicc_maps[m], key=lambda x: x[1])
    }
    for m in list_maps[:-1]
}

location_values = []
for value in dicc_maps["seeds"]:
    for m in list_maps[:-1]:
        value = get_destination(value, dicc_origin_ranges[m])
    location_values.append(value)
print(min(location_values))

35


# Part 2

In [76]:
dicc_origin_to_dest = {
    m: {
            pd.Interval(r[1], r[1] + r[2] -1, closed="both"): r[0] - r[1]
            for r in sorted(dicc_maps[m], key=lambda x: x[1])
    }
    for m in list_maps[:-1]
}

def get_destinations(origin_intervals, dest_intervals):
    left_limit = min(dest_intervals).left
    right_limit = max(dest_intervals).right
    final_output = []
    for origin_interval in sorted(origin_intervals):
        
        if not any(origin_interval.overlaps(d) for d in dest_intervals):
            final_output.append(origin_interval)
            continue
        
        o_l, o_r = origin_interval.left, origin_interval.right
        if (o_l < left_limit) and (o_r >= left_limit):
            final_output.append(pd.Interval(o_l, left_limit - 1, closed='both'))
            
        if (o_r > right_limit) and (o_l <= right_limit):
            final_output.append(pd.Interval(right_limit + 1, o_r, closed='both'))
        
        for dest_interval, v in sorted(dest_intervals.items()):
            if not origin_interval.overlaps(dest_interval):
                continue
            
            d_l, d_r = dest_interval.left, dest_interval.right
            if o_l <= d_l:
                if d_l <= o_r:
                    final_output.append(pd.Interval(d_l, o_r, closed="both") + v)
                else:
                    final_output.append(pd.Interval(d_l, d_r, closed="both") + v)
            else:
                if o_r <= d_r:
                    final_output.append(pd.Interval(o_l, o_r, closed="both") + v)
                else:
                    final_output.append(pd.Interval(o_l, d_r, closed="both") + v)
    return final_output

locations = []
history = []
for i in range(0, len(dicc_maps["seeds"]), 2):
    seed_init = dicc_maps["seeds"][i]
    seed_end = dicc_maps["seeds"][i + 1]
    origin_intervals = [pd.Interval(seed_init, seed_init + seed_end-1, closed="both")]
    history.append({"seeds": origin_intervals})
    for m in list_maps[:-1]:
        origin_intervals = get_destinations(origin_intervals, dicc_origin_to_dest[m])
        history[-1][m] = origin_intervals

    locations += origin_intervals

In [77]:
history

[{'seeds': [Interval(79, 92, closed='both')],
  'seed-to-soil map': [Interval(81, 94, closed='both')],
  'soil-to-fertilizer map': [Interval(81, 94, closed='both')],
  'fertilizer-to-water map': [Interval(81, 94, closed='both')],
  'water-to-light map': [Interval(74, 87, closed='both')],
  'light-to-temperature map': [Interval(78, 80, closed='both'),
   Interval(45, 55, closed='both')],
  'temperature-to-humidity map': [Interval(46, 56, closed='both'),
   Interval(78, 80, closed='both')],
  'humidity-to-location map': [Interval(46, 55, closed='both'),
   Interval(60, 60, closed='both'),
   Interval(82, 84, closed='both')]},
 {'seeds': [Interval(55, 67, closed='both')],
  'seed-to-soil map': [Interval(57, 69, closed='both')],
  'soil-to-fertilizer map': [Interval(57, 69, closed='both')],
  'fertilizer-to-water map': [Interval(61, 69, closed='both'),
   Interval(53, 56, closed='both')],
  'water-to-light map': [Interval(46, 49, closed='both'),
   Interval(54, 62, closed='both')],
  'ligh

In [60]:
dicc_maps["seeds"]

[1044452533,
 40389941,
 3710737290,
 407166728,
 1552449232,
 639689359,
 3327654041,
 26912583,
 3440484265,
 219136668,
 1126550158,
 296212400,
 2332393052,
 229950158,
 200575068,
 532702401,
 4163696272,
 44707860,
 3067657312,
 45353528]

In [74]:
min(locations).left

0

In [43]:
len(locations)

255

In [71]:
for h in history:
    for pos, i in enumerate(h["humidity-to-location map"]):
        if i.left==0:
            print(h, i)

{'seeds': [Interval(3440484265, 3659620932, closed='both')], 'seed-to-soil map': [Interval(3241310824, 3460447491, closed='both')], 'soil-to-fertilizer map': [Interval(3241310824, 3460447491, closed='both')], 'fertilizer-to-water map': [Interval(1798279782, 1964888853, closed='both'), Interval(2318899828, 2371427423, closed='both')], 'water-to-light map': [Interval(1295122702, 1461731773, closed='both'), Interval(493194803, 510500648, closed='both'), Interval(0, 35221749, closed='both')], 'light-to-temperature map': [Interval(23531400, 58753149, closed='both'), Interval(1351062661, 1375359866, closed='both'), Interval(2477361207, 2494667052, closed='both'), Interval(1067305596, 1079941856, closed='both'), Interval(2923124195, 3077097005, closed='both'), Interval(1487998805, 1517714013, closed='both'), Interval(1079941857, 1093889924, closed='both')], 'temperature-to-humidity map': [Interval(23531400, 58753149, closed='both'), Interval(1067305596, 1079941856, closed='both'), Interval(10

In [68]:
dicc_origin_to_dest

{'seed-to-soil map': {Interval(0, 3525157, closed='both'): 1949989349,
  Interval(3525158, 563731935, closed='both'): 1382285602,
  Interval(563731936, 567703746, closed='both'): 1382285602,
  Interval(567703747, 1470577054, closed='both'): -567703747,
  Interval(1470577055, 1532707248, closed='both'): -567703747,
  Interval(1532707249, 1653302078, closed='both'): 618997525,
  Interval(1653302079, 1808056937, closed='both'): 2451323959,
  Interval(1808056938, 2006247204, closed='both'): 145457569,
  Interval(2006247205, 2115309655, closed='both'): 1714863691,
  Interval(2115309656, 2389762346, closed='both'): 1714863691,
  Interval(2389762347, 2406371823, closed='both'): -1424758845,
  Interval(2406371824, 2471473044, closed='both'): -1424758845,
  Interval(2471473045, 3920284336, closed='both'): -199173441,
  Interval(3920284337, 4259380896, closed='both'): -2873570137},
 'soil-to-fertilizer map': {Interval(0, 297180571, closed='both'): 131659986,
  Interval(297180572, 342253284, clos

In [13]:
min(dicc_origin_to_dest["seed-to-soil map"]).left

0

In [14]:
max(dicc_origin_to_dest["seed-to-soil map"]).right

4259380896

In [ ]:
#Too high
2784810021
1457768254

#too low
405301